In [2]:
#r "nuget:Microsoft.DotNet.Interactive.AI, *-*"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-eng/nuget/v3/index.json Installed Packages Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23319.1

Loading extensions from `C:\Users\dicolomb\.nuget\packages\skiasharp\2.88.5\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

Loading extension script from `C:\Users\dicolomb\.nuget\packages\microsoft.dotnet.interactive.ai\1.0.0-beta.23319.1\interactive-extensions\dotnet\extension.dib`

Added magic command `#!connect openai`.

In [2]:
#!value --name key
YOUR AZURE OPEN AI KEY

In [3]:
#!value --name endpoint
https://your-enpoint.openai.azure.com/

In [4]:
#!connect azure-openai --model-type TextEmbeddingGenerator --kernel-name knowledge --api-key @value:key --endpoint @value:endpoint --deployment text-embedding-ada-002

Kernel added: #!knowledge(duckDbVectorRAW)

Kernel added: #!knowledge(duckDbVector)

Kernel added: #!knowledge

In [5]:
#!connect azure-openai --model-type TextCompletion --kernel-name textCompletion --api-key @value:key --endpoint @value:endpoint --deployment text-davinci-003 --use-knowledge knowledge

Kernel added: #!textCompletion(skill)

Kernel added: #!textCompletion

In [7]:
using System;
using System.Globalization;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.SkillDefinition;

public class MathSkill
{
    /// <summary>
    /// Returns the Addition result of initial and amount values provided.
    /// </summary>
    /// <param name="initialValueText">Initial value as string to add the specified amount</param>
    /// <param name="context">Contains the context to get the numbers from</param>
    /// <returns>The resulting sum as a string.</returns>
    [SKFunction(),]
    [SKName("Add")]
    [SKParameter("Amount","Amount to add")]
    public Task<string> AddAsync(string input, SKContext context)
    {
        if (!int.TryParse(input.Replace("\"", string.Empty), NumberStyles.Any, CultureInfo.InvariantCulture, out var initialValue))
        {
            return Task.FromException<string>(new ArgumentOutOfRangeException(
                nameof(input), input, "Initial value provided is not in numeric format"));
        }

        string contextAmount = context["Amount"].Replace("\"", string.Empty);
        if (!int.TryParse(contextAmount, NumberStyles.Any, CultureInfo.InvariantCulture, out var amount))
        {
            return Task.FromException<string>(new ArgumentOutOfRangeException(
                nameof(context), contextAmount, "Context amount provided is not in numeric format"));
        }

        var result = initialValue + amount;

        return Task.FromResult(result.ToString(CultureInfo.InvariantCulture));
    }    
}

In [8]:
var mathSkill = new MathSkill();

In [9]:
#!set --value @csharp:mathSkill --name mathSkill --byref

In [11]:
#!function doMath
{{mathSkill.Add $input}}


In [ ]:
#!set --name Amount --value 5
#!use-skills function._GLOBAL_FUNCTIONS_.doMath
37